In [32]:
import os
import subprocess
from datetime import datetime
from google.colab import drive

# === CẤU HÌNH RIÊNG (CHỈ THAY 1 LẦN) ===
USERNAME   = "FongNgoo"                                   # ← GitHub username
REPO_NAME  = "Basic_Dynamic_Prices_base_on_Demand_Model" # ← Tên repo
BRANCH     = "main"                                       # hoặc "master"
REPO_PATH  = f"/content/drive/MyDrive/Colab_Notebooks/{REPO_NAME}"
SSH_DIR    = "/content/drive/MyDrive/.colab_ssh"          # Nơi lưu key vĩnh viễn
# ======================================

REPO_DIR = f"/content/{REPO_NAME}"
REPO_URL = f"git@github.com:{USERNAME}/{REPO_NAME}.git"


In [33]:
def setup_permanent_ssh():
    """Chạy 1 lần duy nhất → tạo và lưu SSH key vĩnh viễn trên Drive"""
    drive.mount('/content/drive', force_remount=True)
    os.makedirs(SSH_DIR, exist_ok=True)

    key_path = f"{SSH_DIR}/id_colab"

    if os.path.exists(key_path):
        print("SSH key đã tồn tại trên Drive → sử dụng lại.")
    else:
        print("Đang tạo SSH key vĩnh viễn (lưu trên Drive)...")
        !ssh-keygen -t rsa -b 4096 -C "colab-permanent@$(date +%s)" -f "{key_path}" -N "" -q
        print("ĐÃ TẠO SSH KEY MỚI!")

    print("\n" + "="*70)
    print("COPY PUBLIC KEY DƯỚI ĐÂY VÀ DÁN VÀO: https://github.com/settings/ssh/new")
    print("="*70)
    !cat "{key_path}.pub"
    print("="*70)
    print("Sau khi dán xong → chạy load_ssh() để kích hoạt trong runtime này")
    print("Chỉ cần làm 1 lần duy nhất là xong mãi mãi!")

In [34]:
def load_ssh():
    """Tải SSH key từ Drive vào runtime (chạy mỗi khi mở notebook mới)"""
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive', force_remount=True)

    key_path = f"{SSH_DIR}/id_colab"
    if not os.path.exists(key_path):
        raise FileNotFoundError("Chưa có SSH key! Chạy setup_permanent_ssh() trước.")

    !mkdir -p /root/.ssh
    !cp "{key_path}" /root/.ssh/id_colab
    !cp "{key_path}.pub" /root/.ssh/id_colab.pub
    !chmod 600 /root/.ssh/id_colab
    !chmod 644 /root/.ssh/id_colab.pub

    # Cấu hình SSH dùng key này
    config = f"""
Host github.com
    HostName github.com
    User git
    IdentityFile /root/.ssh/id_colab
    StrictHostKeyChecking no
    """
    with open('/root/.ssh/config', 'w') as f:
        f.write(config.strip())

    # Fix host key
    !ssh-keyscan github.com >> /root/.ssh/known_hosts 2>/dev/null

    # Test kết nối
    print("Đang test SSH với GitHub...")
    result = subprocess.run(['ssh', '-T', 'git@github.com'], capture_output=True, text=True)
    if "successfully authenticated" in result.stderr:
        print("SSH KẾT NỐI THÀNH CÔNG! Hi {USERNAME}!")
    else:
        print("SSH chưa được add vào GitHub → chạy setup_permanent_ssh() để lấy public key")

In [35]:
def ensure_repo():
    """Clone repo nếu chưa có"""
    if not os.path.exists(REPO_PATH):
        print(f"Đang clone repo {USERNAME}/{REPO_NAME}...")
        !git clone f"git@github.com:{USERNAME}/{REPO_NAME}.git" "{REPO_PATH}"
    os.chdir(REPO_PATH)
    print(f"Đã vào: {REPO_PATH}")

In [43]:
def push(message=None, force=False):
    """Chỉ push – nhanh, gọn, không pull"""
    load_ssh()           # ← Tự động tải key
    ensure_repo()        # ← Vào đúng thư mục

    status = subprocess.check_output(['git', 'status', '--porcelain']).decode().strip()
    if not status:
        print("Không có thay đổi nào.")
        return

    print(f"Phát hiện thay đổi:\n{status}")

    msg = message or f"Auto push từ Colab - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
    print(f"Commit: {msg}")
    !git add . --all
    !git commit -m "{msg}" || echo "Không có gì để commit"

    if force:
        !git push origin {BRANCH} --force-with-lease
        print("FORCE PUSH THÀNH CÔNG!")
    else:
        result = subprocess.run(['git', 'push', 'origin', BRANCH], capture_output=True, text=True)
        if result.returncode != 0:
            print("Push bị chặn → tự động force-with-lease")
            !git push origin {BRANCH} --force-with-lease
        else:
            print("PUSH THÀNH CÔNG!")

In [39]:
def delete_all_ssh_keys_and_reset():
    """
    XÓA SẠCH HOÀN TOÀN MỌI SSH KEY CŨ
    → Dùng khi muốn tạo key hoàn toàn mới, không bị lẫn key cũ
    """
    print("ĐANG XÓA SẠCH TẤT CẢ SSH KEY CŨ...\n")

    # 1. Xóa key trên Google Drive (nơi lưu vĩnh viễn)
    if os.path.exists(SSH_DIR):
        !rm -rf "{SSH_DIR}"
        print(f"ĐÃ XÓA THƯ MỤC KEY TRÊN DRIVE: {SSH_DIR}")
    else:
        print(f"Không tìm thấy thư mục key trên Drive: {SSH_DIR}")

    # 2. Xóa key trong runtime hiện tại
    !rm -rf /root/.ssh
    print("ĐÃ XÓA TẤT CẢ KEY TRONG RUNTIME (/root/.ssh)")

    # 3. Xóa remote SSH cũ trong repo (nếu có)
    if os.path.exists(REPO_PATH):
        os.chdir(REPO_PATH)
        current_url = subprocess.check_output(['git', 'remote', 'get-url', 'origin'], text=True).strip()
        if "git@github.com" in current_url:
            print(f"Remote hiện tại: {current_url}")
            print("Đang chuyển tạm về HTTPS để tránh lỗi khi key bị xóa...")
            !git remote set-url origin "https://github.com/{USERNAME}/{REPO_NAME}.git"

    print("\nHOÀN TẤT XÓA SẠCH!")
    print("Bây giờ bạn có thể chạy lại:")
    print("   setup_permanent_ssh()   ← để tạo key mới + copy public key mới")
    print("   Sau khi dán key mới vào GitHub → dùng load_ssh() + push() bình thường")

# ====================== CÁCH DÙNG KHI MUỐN TẠO KEY MỚI ======================
# delete_all_ssh_keys_and_reset()    # ← xóa sạch mọi thứ
# setup_permanent_ssh()              # ← tạo key mới + copy public key mới dán vào GitHub
# =======================================================================

In [41]:
delete_all_ssh_keys_and_reset()
setup_permanent_ssh()

ĐANG XÓA SẠCH TẤT CẢ SSH KEY CŨ...

Không tìm thấy thư mục key trên Drive: /content/drive/MyDrive/.colab_ssh
ĐÃ XÓA TẤT CẢ KEY TRONG RUNTIME (/root/.ssh)

HOÀN TẤT XÓA SẠCH!
Bây giờ bạn có thể chạy lại:
   setup_permanent_ssh()   ← để tạo key mới + copy public key mới
   Sau khi dán key mới vào GitHub → dùng load_ssh() + push() bình thường
Mounted at /content/drive
Đang tạo SSH key vĩnh viễn (lưu trên Drive)...
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
ĐÃ TẠO SSH KEY MỚI!

COPY PUBLIC KEY DƯỚI ĐÂY VÀ DÁN VÀO: https://github.com/settings/ssh/new
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQCrKjalDW4A0L0vP+/tM5hqo+PTch+xRUSITD7tu3CbXK8C4I3ZdoLI2FtX3qX916h43xlljmzPQv/D/aD3Wl+oVJpMFEG+E25/WgiZbToQXzYU1JOBmv4Rx3F6NTS5Bgl/Mf3Eb0ea91oMoJnacdvEWQJWrv+fUOjnmzn8rkLy9uAGoiYjW/S/FPg8hEOL9vs5VoqYQ9T

In [44]:
load_ssh()
push()

Đang test SSH với GitHub...
SSH KẾT NỐI THÀNH CÔNG! Hi {USERNAME}!
Đang test SSH với GitHub...
SSH KẾT NỐI THÀNH CÔNG! Hi {USERNAME}!
Đã vào: /content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model
Phát hiện thay đổi:
M AUTO_SYNC_COLAB_GITHUB_SSH.ipynb
 M Basic_Dynamic_Prices_base_on_Demand_Model
Commit: Auto push từ Colab - 2025-11-17 18:45:05
[main 00d71de] Auto push từ Colab - 2025-11-17 18:45:05
 1 file changed, 1 insertion(+), 1 deletion(-)
Push bị chặn → tự động force-with-lease
fatal: could not read Username for 'https://github.com': No such device or address


In [45]:
# FIX CUỐI CÙNG: ÉP GIT DÙNG SSH THAY VÌ HTTPS

%cd /content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model

# 1. Ép remote về SSH (quan trọng nhất)
!git remote set-url origin git@github.com:FongNgoo/Basic_Dynamic_Prices_base_on_Demand_Model.git

# 2. Kiểm tra lại remote (phải thấy dòng git@github.com)
print("Remote hiện tại:")
!git remote -v

# 3. Push ngay và luôn (lần này 100% thành công)
!git push origin main --force-with-lease

print("\nHOÀN TẤT! TỪ GIỜ TRỞ ĐI CHỈ CẦN GỌI push() HOẶC sync() LÀ BAY NGON LÀNH!")

/content
Remote hiện tại:
origin	git@github.com:FongNgoo/Basic_Dynamic_Prices_base_on_Demand_Model.git (fetch)
origin	git@github.com:FongNgoo/Basic_Dynamic_Prices_base_on_Demand_Model.git (push)
Enumerating objects: 50, done.
Counting objects: 100% (50/50), done.
Delta compression using up to 2 threads
Compressing objects: 100% (40/40), done.
Writing objects: 100% (40/40), 1.34 MiB | 5.81 MiB/s, done.
Total 40 (delta 22), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (22/22), completed with 4 local objects.
To github.com:FongNgoo/Basic_Dynamic_Prices_base_on_Demand_Model.git
   7fb6248..00d71de  main -> main

HOÀN TẤT! TỪ GIỜ TRỞ ĐI CHỈ CẦN GỌI push() HOẶC sync() LÀ BAY NGON LÀNH!
